In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/Project_HonesDoctor2020

/content/drive/My Drive/Project_HonesDoctor2020


In [3]:
!pip install googletrans
!pip install tqdm #--upgrade
!pip install twython

  Created wheel for googletrans: filename=googletrans-2.4.0-cp36-none-any.whl size=15777 sha256=3f161dc18d4a337b6b101ea62668a4edc751ec61f2b33712f5e98f2b27d61a2d
  Stored in directory: /root/.cache/pip/wheels/50/d6/e7/a8efd5f2427d5eb258070048718fa56ee5ac57fd6f53505f95
Successfully built googletrans


In [0]:
import requests
from bs4 import BeautifulSoup
from time import  time, sleep
from random import randint

from googletrans import Translator

from tqdm import tqdm
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from nltk import sent_tokenize, word_tokenize
from nltk.sentiment.vader import  SentimentIntensityAnalyzer
from nltk.sentiment.util import mark_negation

In [5]:
import nltk

nltk.download("punkt")
nltk.download("vader_lexicon")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [0]:
def honestdoc_comment(url):
  """
    INPUT
      url : String
        URL of the target website
    OUTPUT
      comment : List
        List of comments
      score : List
        List of rating score
  """
  data = requests.get(url)
  print("requests code : {}".format(data.status_code))
  print("note\n2xx : successs\n4xx, 5xx : error")

  start_time = time()
  r = requests.get(url, params=dict(query="web scraping", page=1))
  soup = BeautifulSoup(r.text, "html.parser")
  n = len(soup.find_all("div", {"class": "comments__content"}))

  comment = [ soup.find_all("div", {"class": "comments__content"})[i].get_text().strip() for i in range(0, n) ]
  score = [ soup.find_all("span", {"class": "stars star-rating"})[i].attrs["data-score"] for i in range(0, n) ]

  elapsed_time = time() - start_time
  print("Time used for scraping data from page 1 : {} s".format(elapsed_time))
  sleep(randint(1,3))

  p = 2
  while n > 0:
    start_time = time()
    r = requests.get(url, params=dict(query="web scraping", page=p))
    soup = BeautifulSoup(r.text, "html.parser")
    n = len(soup.find_all("div", {"class": "comments__content"}))
    [ comment.append(soup.find_all("div", {"class": "comments__content"})[i].get_text().strip()) for i in range(0,n) ]
    [ score.append(soup.find_all("span", {"class": "stars star-rating"})[i].attrs["data-score"]) for i in range(0,n) ]
    elapsed_time = time() - start_time
    print("Time used for scraping data from page {} : {} s".format(p, elapsed_time))
    p += 1
    sleep(randint(1,3))
  #backup
  pd.DataFrame({"comment": comment, "score": score}).to_csv("comment_"+str(url[url.rfind("/")+1:]) + ".csv", index=False)
  return comment, score


In [0]:
# We choose the following lists of hospitals to make sentiment analysis.
# โรงพยาบาลศิริราช ปิยมหาราชการุณย์
# https://www.honestdocs.co/hospitals/siriraj-piyamaharajkarun-hospital
# โรงพยาบาลสิรินธร
# https://www.honestdocs.co/hospitals/sirindhorn-hospital
# โรงพยาบาลตำรวจ
# https://www.honestdocs.co/hospitals/police-general-hospital

In [7]:
#โรงพยาบาลศิริราช ปิยมหาราชการุณย์
siripiya_comments, siripiya_scores = honestdoc_comment(r"https://www.honestdocs.co/hospitals/siriraj-piyamaharajkarun-hospital")

requests code : 200
note
2xx : successs
4xx, 5xx : error
Time used for scraping data from page 1 : 0.9917488098144531 s
Time used for scraping data from page 2 : 0.8928046226501465 s
Time used for scraping data from page 3 : 0.9313454627990723 s
Time used for scraping data from page 4 : 0.9505720138549805 s
Time used for scraping data from page 5 : 0.9117028713226318 s
Time used for scraping data from page 6 : 0.9851694107055664 s
Time used for scraping data from page 7 : 1.0098674297332764 s
Time used for scraping data from page 8 : 1.0239372253417969 s
Time used for scraping data from page 9 : 0.908318042755127 s
Time used for scraping data from page 10 : 0.897758960723877 s
Time used for scraping data from page 11 : 0.9117190837860107 s
Time used for scraping data from page 12 : 1.0033259391784668 s
Time used for scraping data from page 13 : 0.8898611068725586 s
Time used for scraping data from page 14 : 0.8985157012939453 s
Time used for scraping data from page 15 : 0.9128623008728

In [13]:
siripiya_comments

['วันนี้ไปเยี่ยมเพื่อนมาคะพึ่งคลอดลูก เป็นห้องรวมแต่พยาบาลดูแลดีมาก หมอทำคลอดดีมากต้องขอชื่นชมค่ะ แถมสถานที่สะอาดมากๆมีที่ให้ญาติๆได้เข้าเยี่ยมได้อย่างสะดวกสะบาย ถึงแม้จะเป็นห้องรวมก็ตาม แต่รู้สึกโอเคกับการบริการในครั้งนี้มาก หวังว่าเพื่อนของฉันจะได้รับการบริการที่ดีแบบนี้จนกว่าจะได้กลับมาผักผ่อนที่บ้านนะคะ',
 'ผมเคยปัสสาวะบ่อยมากๆ วันนึง 20รอบ \r\n แล้วได้เข้ารับการรักษาที่โรงพยาบาลแห่งนี้  ผมประทับใจในการบริการของคุณหมอและเจ้าหน้าที่มากๆ รับฟังคนไข้ดีมาก  ดูแลดี ยิ้มแย้ม   ขั้นตอนต่างๆดูเป็นมืออาชีพมากๆมีระดับ รวดเร็วมากๆครับ  ประทับใจมากครับ  รู้สึกไม่ผิดหวังที่เข้ารับการรักษาที่โรงพยาบาลแห่งนี้\r\n\r\nและชอบการdesign ตึก/ภายในตึกหรือ ห้องต่างๆในโรงพยาบาลนี้มากครับ ดูสวยงามสะอาดตา  ทันสมัยมากๆ',
 'ดีมากเลยค่ะะ เคยไม่สบายเป็นไข้คุณหมอพูดจาน่ารัก พยาบาลดีมากเลยค่ะ เป็นโรงพยาบาลที่ดีในระแวกใกล้บ้านที่หนึ่งเลยค่ะ ค่ารักษาเหมาะสมกับการเข้ารับการรักษา และผู้จ่ายยามีการอธิบายการรับประทานยาชัดเจน มีความติดตามคนไข้อยู่ตลอด คุณหมอคอยตามอาการคนไข้อย่างใกล้ชิด และมีอัธยาศัยดี ยิ้มแย้ม มีการให้ค

In [9]:
siripiya = pd.read_csv("comment_siriraj-piyamaharajkarun-hospital.csv")
print(siripiya.shape)
siripiya.head()

(396, 2)


,comment,score
0,วันนี้ไปเยี่ยมเพื่อนมาคะพึ่งคลอดลูก เป็นห้องรว...,5
1,ผมเคยปัสสาวะบ่อยมากๆ วันนึง 20รอบ \r\n แล้วได้...,4
2,ดีมากเลยค่ะะ เคยไม่สบายเป็นไข้คุณหมอพูดจาน่ารั...,5
3,แม่ผมเข้ารับการรักษาอาการเส้นเลือดสมองตีบและเป...,1
4,ดีมากเลยค่ะะ เคยไม่สบายเป็นไข้คุณหมอพูดจาน่ารั...,5


In [0]:
'''
gt = Translator()
input_text = "หิวข้าวรึยัง"
gt.translate(input_text, src="th", dest="en").text
'''

In [0]:
def th2en(comment):
  print(comment + "\n")
  return Translator().translate(comment, src="th", dest="en").text

In [0]:
#for check lenght 2000 with issue comment

i = 0
for x in siripiya['comment']:
 if len(siripiya['comment'][i]) > 2000:
  print(len(siripiya['comment'][i]), "-> index : ", i)
 i=i+1


In [0]:
#check emoji
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [13]:
'''
siripiya["en"] = siripiya.progress_apply(lambda x: th2en(x["comment"]), axis=1)
There is an unexpected issue at comment 395th  - index 394 - Found Emoji
395/396 [05:52<00:00,  1.11it/s]เคยไปหาแพทย์ผิวหนัง คุณหมอต่อพงษ์ (จำนามสกุลไม่ได้อะค่ะ) การบริการดีเยี่ยม👍🏻 คุณหมอให้คำปรึกษาดี ติดตามอาการ แต่ควรนัดล่วงหน้านะค่ะ จะได้ไม่ต้องรอนาน^^ ข้อดีของที่นี่ สถานที่⭐️⭐️⭐️⭐️⭐️ ที่จอดรถมีเยอะค่ะ ถ้ามาหาคุณหมอจอดฟรี ถ้าไม่ได้มาหาคุณหมอชม.100฿😅นะจ๊ะ ข้อเสีย รถติดมากจ้า🚗🚕🚙 ค่ารักษาที่นี่พอๆกับรพ.เอกชน (เพราะรายได้ส่วนหนึ่งนำไปช่วยเหลือผู้ป่วยขาดแคลนจ้า)
so try to remove emoji and run above again
'''
print(siripiya.iloc[394,:].values[0])
siripiya.iloc[394,0] = remove_emojis(siripiya.iloc[394,0])
siripiya.iloc[394,:].values[0]

เคยไปหาแพทย์ผิวหนัง คุณหมอต่อพงษ์ (จำนามสกุลไม่ได้อะค่ะ) การบริการดีเยี่ยม👍🏻 คุณหมอให้คำปรึกษาดี ติดตามอาการ แต่ควรนัดล่วงหน้านะค่ะ จะได้ไม่ต้องรอนาน^^ ข้อดีของที่นี่ สถานที่⭐️⭐️⭐️⭐️⭐️ ที่จอดรถมีเยอะค่ะ ถ้ามาหาคุณหมอจอดฟรี ถ้าไม่ได้มาหาคุณหมอชม.100฿😅นะจ๊ะ ข้อเสีย รถติดมากจ้า🚗🚕🚙 ค่ารักษาที่นี่พอๆกับรพ.เอกชน (เพราะรายได้ส่วนหนึ่งนำไปช่วยเหลือผู้ป่วยขาดแคลนจ้า)


'เคยไปหาแพทย์ผิวหนัง คุณหมอต่อพงษ์ (จำนามสกุลไม่ได้อะค่ะ) การบริการดีเยี่ยม คุณหมอให้คำปรึกษาดี ติดตามอาการ แต่ควรนัดล่วงหน้านะค่ะ จะได้ไม่ต้องรอนาน^^ ข้อดีของที่นี่ สถานที่ ที่จอดรถมีเยอะค่ะ ถ้ามาหาคุณหมอจอดฟรี ถ้าไม่ได้มาหาคุณหมอชม.100฿นะจ๊ะ ข้อเสีย รถติดมากจ้า ค่ารักษาที่นี่พอๆกับรพ.เอกชน (เพราะรายได้ส่วนหนึ่งนำไปช่วยเหลือผู้ป่วยขาดแคลนจ้า)'

In [14]:
tqdm.pandas()

siripiya["en"] = siripiya.progress_apply(lambda x: th2en(x["comment"]), axis=1)

  1%|          | 2/396 [00:00<00:20, 19.15it/s]

วันนี้ไปเยี่ยมเพื่อนมาคะพึ่งคลอดลูก เป็นห้องรวมแต่พยาบาลดูแลดีมาก หมอทำคลอดดีมากต้องขอชื่นชมค่ะ แถมสถานที่สะอาดมากๆมีที่ให้ญาติๆได้เข้าเยี่ยมได้อย่างสะดวกสะบาย ถึงแม้จะเป็นห้องรวมก็ตาม แต่รู้สึกโอเคกับการบริการในครั้งนี้มาก หวังว่าเพื่อนของฉันจะได้รับการบริการที่ดีแบบนี้จนกว่าจะได้กลับมาผักผ่อนที่บ้านนะคะ

ผมเคยปัสสาวะบ่อยมากๆ วันนึง 20รอบ 
 แล้วได้เข้ารับการรักษาที่โรงพยาบาลแห่งนี้  ผมประทับใจในการบริการของคุณหมอและเจ้าหน้าที่มากๆ รับฟังคนไข้ดีมาก  ดูแลดี ยิ้มแย้ม   ขั้นตอนต่างๆดูเป็นมืออาชีพมากๆมีระดับ รวดเร็วมากๆครับ  ประทับใจมากครับ  รู้สึกไม่ผิดหวังที่เข้ารับการรักษาที่โรงพยาบาลแห่งนี้

และชอบการdesign ตึก/ภายในตึกหรือ ห้องต่างๆในโรงพยาบาลนี้มากครับ ดูสวยงามสะอาดตา  ทันสมัยมากๆ



  1%|          | 4/396 [00:00<00:33, 11.55it/s]

ดีมากเลยค่ะะ เคยไม่สบายเป็นไข้คุณหมอพูดจาน่ารัก พยาบาลดีมากเลยค่ะ เป็นโรงพยาบาลที่ดีในระแวกใกล้บ้านที่หนึ่งเลยค่ะ ค่ารักษาเหมาะสมกับการเข้ารับการรักษา และผู้จ่ายยามีการอธิบายการรับประทานยาชัดเจน มีความติดตามคนไข้อยู่ตลอด คุณหมอคอยตามอาการคนไข้อย่างใกล้ชิด และมีอัธยาศัยดี ยิ้มแย้ม มีการให้ความรู้ในด้านต่างๆเพิ่มขึ้นมาอีก โรงพยาบาลมีความสะอาด มีสุขอนามัยที่ดี มีบริการที่ดี

แม่ผมเข้ารับการรักษาอาการเส้นเลือดสมองตีบและเป็นผู้ป่วยติดเตียงที่โรงพยาบาลเป็นเวลา2เดือน และใน3สัปดาห์หลังพักฟื้นในหอพักผู้ป่วยชั้น10โซนC ในขณะที่ผมเฝ้าไข้ เวลาแม่มีไข้หมอเข้ามาเยี่ยมไข้ก็จะถามพยาบาลว่าผู้ป่วยมีแผลตรงไหนไหม พยาบาลก็ตอบว่าไม่มี จนวันที่22ต.ค.2561หมอเจ้าของไข้ให้ผู้ป่วยกลับบ้านได้ เราจึงกลับในบ่ายวันนั้นถึงศูนย์ดูแลผู้ป่วยที่ต่างจังหวัด ผู้ดูแลแม่พบว่าที่ก้นของแม่มีแผลกดทับ ผมตกใจมาก และญาติๆก็ตกใจ ว่าเกิดอะไรขึ้น ในขณะที่อยู่ร.พ.ไม่มีใครเอ่ยถึงว่าแม่มีแผลกดทับ และไม่มีกล่าวถึงการรักษาแผลกดทับเลย มีแต่ประมาณ1สัปดาห์ที่ผ่านมาก่อนออกร.พ.แม่มีไข้และหมอให้ยาฆ่าเชื้อ ในวันที่รู้ว่าแม่มีแผลผมโทรไปถาม(โวย)พยา

  2%|▏         | 6/396 [00:01<02:24,  2.69it/s]

...

ราคาเท่ากับโรงพยาบาลเอกชนระดับกลาง แต่การบริการดีเจ้าหน้าที่ยิ้มแย้มช่วยเหลือดีมาก คุณหมอแนะนำดีมากๆ และมีตัวเลือกในการรักษาพยาบาลได้ภายใต้คำแนะนำของคุณหมอที่ดีที่สุดและไม่มีค่าใช้จ่ายที่แพงเกินไป

ถึงแม้จะเป๊น ร.พ.ขนาดใหญ่แต่ขั้นตอนการใช้บริการเข้าใจง่าย
อัธยาศัยแพทย์-พยาบาล ดีค่

มาครั้งแรกก็รู้สึกดีตั้งแต่ลานจอดรถเลยค่ะ ลานจอดรถกว้างมาก สะดวก สบาย ในรพ.บรรยากาศดี วิวสวย สะอาด ร้านอาหาร ร้านกาแฟ มีให้เลือกมากมาย เจ้าหน้าที่บริการได้รวดเร็วทันใจ พูดจาด้วยน้ำเสียงไพเราะน่าฟัง คุณหมอมีคำแนะนำที่ดี และตรงตามอาการที่เป็น อธิบายเข้าใจง่าย สามารถถามได้ทุกปัญหาสุขภาพ ค่าบริการทางการแพทย์ก็ไม่แพงค่ะ ช่องชำระเงิน และช่องจ่ายยาก็ไม่ต้องรอคิวนาน และยังมีพนักงานรักษาความปลอดภัยเกือบทุกจุด รวมๆแล้วดีมากๆค่ะ



 99%|█████████▉| 394/396 [00:47<00:00,  9.69it/s]

ที่จอดรถสะดวก

เคยไปหาแพทย์ผิวหนัง คุณหมอต่อพงษ์ (จำนามสกุลไม่ได้อะค่ะ) การบริการดีเยี่ยม คุณหมอให้คำปรึกษาดี ติดตามอาการ แต่ควรนัดล่วงหน้านะค่ะ จะได้ไม่ต้องรอนาน^^ ข้อดีของที่นี่ สถานที่ ที่จอดรถมีเยอะค่ะ ถ้ามาหาคุณหมอจอดฟรี ถ้าไม่ได้มาหาคุณหมอชม.100฿นะจ๊ะ ข้อเสีย รถติดมากจ้า ค่ารักษาที่นี่พอๆกับรพ.เอกชน (เพราะรายได้ส่วนหนึ่งนำไปช่วยเหลือผู้ป่วยขาดแคลนจ้า)



100%|██████████| 396/396 [00:49<00:00,  8.06it/s]

คุณหมอเป็นอาจารย์แพทย์ ไม่ต้องกังวลว่าจะไปเจอหมอจบใหม่ไม่มีประสบการณ์ สถานที่กว้างขวางสะอาด ราคายอมรับได้



In [15]:
siripiya.head(10)

,comment,score,en
0,วันนี้ไปเยี่ยมเพื่อนมาคะพึ่งคลอดลูก เป็นห้องรว...,5,Today I visited my friend's baby. But as well ...
1,ผมเคยปัสสาวะบ่อยมากๆ วันนึง 20รอบ \r\n แล้วได้...,4,I urinate very often around day 20.\r\n He was...
2,ดีมากเลยค่ะะ เคยไม่สบายเป็นไข้คุณหมอพูดจาน่ารั...,5,"Good Ceaa had discomfort, fever, doctor's parl..."
3,แม่ผมเข้ารับการรักษาอาการเส้นเลือดสมองตีบและเป...,1,I've been treating cerebral thrombosis and pat...
4,ดีมากเลยค่ะะ เคยไม่สบายเป็นไข้คุณหมอพูดจาน่ารั...,5,"Good Ceaa had discomfort, fever, doctor's parl..."
5,เป็นโรงบาลมาตราฐานเอกชน สะอาด ดูปลอดโปร่งกว่าศ...,5,A standard private hospitals clean and clear v...
6,บริการดี รวดเร็ว และทันสมัย สะดวกและรวดเร็ว ด...,5,Good service and modern convenience and good c...
7,เป็นโรงบาลมาตราฐานเอกชน สะอาด ดูปลอดโปร่งกว่าศ...,5,A standard private hospitals clean and clear v...
8,บริการดี รวดเร็ว สถานที่สะอาด มีที่จอดรถสะดวกส...,5,"Fast service, good clean place with parking fa..."
9,ทำการตรวจรักษาเร็ว ราคาไม่แพงมาก บริการดี อธิบ...,4,Please heal faster More affordable services to...


In [16]:
rename_col = ['comment','en','score']
siripiya = siripiya.reindex(columns=rename_col)
siripiya.columns = ['th_comment','en_comment','score']
siripiya.head()

,th_comment,en_comment,score
0,วันนี้ไปเยี่ยมเพื่อนมาคะพึ่งคลอดลูก เป็นห้องรว...,Today I visited my friend's baby. But as well ...,5
1,ผมเคยปัสสาวะบ่อยมากๆ วันนึง 20รอบ \r\n แล้วได้...,I urinate very often around day 20.\r\n He was...,4
2,ดีมากเลยค่ะะ เคยไม่สบายเป็นไข้คุณหมอพูดจาน่ารั...,"Good Ceaa had discomfort, fever, doctor's parl...",5
3,แม่ผมเข้ารับการรักษาอาการเส้นเลือดสมองตีบและเป...,I've been treating cerebral thrombosis and pat...,1
4,ดีมากเลยค่ะะ เคยไม่สบายเป็นไข้คุณหมอพูดจาน่ารั...,"Good Ceaa had discomfort, fever, doctor's parl...",5


In [0]:
#export to csv incluing th-comment, en-comment, score
siripiya.to_csv('en-comment_siriraj-piya-hospital.csv', index=False ,encoding='utf-8')

In [18]:
en_siripiya = pd.read_csv("en-comment_siriraj-piya-hospital.csv")
print(en_siripiya.shape)
en_siripiya.head()

(396, 3)


,th_comment,en_comment,score
0,วันนี้ไปเยี่ยมเพื่อนมาคะพึ่งคลอดลูก เป็นห้องรว...,Today I visited my friend's baby. But as well ...,5
1,ผมเคยปัสสาวะบ่อยมากๆ วันนึง 20รอบ \r\n แล้วได้...,I urinate very often around day 20.\r\n He was...,4
2,ดีมากเลยค่ะะ เคยไม่สบายเป็นไข้คุณหมอพูดจาน่ารั...,"Good Ceaa had discomfort, fever, doctor's parl...",5
3,แม่ผมเข้ารับการรักษาอาการเส้นเลือดสมองตีบและเป...,I've been treating cerebral thrombosis and pat...,1
4,ดีมากเลยค่ะะ เคยไม่สบายเป็นไข้คุณหมอพูดจาน่ารั...,"Good Ceaa had discomfort, fever, doctor's parl...",5
